## HamLib Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies and Initialize Modules

This cell is used to pull in dependencies from standard libraries, the Hamlib helper functions, and the newly created observables module (in WIP).
Initialize the modules so a re-run from the beginning is repeatable.

In [1]:
import numpy as np
from math import sin, cos, pi
import time

# Import Qiskit and Qiskit Pauli operator classes
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit import QuantumCircuit, transpile

# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')

# Configure module paths
import sys
sys.path.insert(1, "_common")
sys.path.insert(1, "qiskit")

# Import HamLib helper functions (from _common)
import hamlib_utils

# Import Observable helper functions
import observables

# Import Hamlib Simulation kernel (from qiskit)
import hamlib_simulation_kernel


### Configure Options for Execution

In [2]:
# Qubit width of the Hamiltonian
num_qubits = 6

# Parameters of Trotterized simulation
K = 1
t = 0.05

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

# for debugging observables module
observables.verbose_circuits = False


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [3]:
# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
# If no name, it means we use an explicitly specified Hamiltonian
hamiltonian_name = 'TFIM'

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }

if hamiltonian_name == 'Heisenberg':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }
    
if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "pbc", "enc": "bk", "U":12 }

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "nonpbc", "enc": "gray", "U":10 }

if hamiltonian_name == 'Max3Sat':
    hamiltonian_params = { "ratio": "2", "rinst": "02" }

if hamiltonian_name == 'chemistry/electronic/standard/H2':
    hamiltonian_params = { "ham_BK": '' }
    
########### Explicit Hamiltonians

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

# extract Hamiltonian from HamLib
if hamiltonian_name != '':

    # load the HamLib file for the given hamiltonian name
    hamlib_utils.load_from_file(filename=hamiltonian_name)

    # return a sparse Pauli list of terms queried from the open HamLib file
    sparse_pauli_terms, dataset_name = hamlib_utils.get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=hamiltonian_params)
    print(f"... sparse_pauli_terms = \n{sparse_pauli_terms}")
    
    # convert the SparsePauliList to a SparsePauliOp object
    sparse_pauli_op = hamlib_simulation_kernel.ensure_sparse_pauli_op(sparse_pauli_terms, num_qubits)
    print(f"... sparse_pauli_op = \n{sparse_pauli_op}")
    print("")
    
    # convert SparsePauliOp to list form (for use by expectation functions below)
    pauli_terms = sparse_pauli_op.to_list()
    
# create explicit Hamiltonian
else:
    sparse_pauli_op = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
    pauli_terms = sparse_pauli_op.to_list()
    qc = None

print(f"... Hamiltonian name = {hamiltonian_name}")
print(pauli_terms)
print("")


... sparse_pauli_terms = 
[({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 5: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({5: 'Z', 4: 'Z'}, (1+0j)), ({2: 'Z', 3: 'Z'}, (1+0j)), ({3: 'Z', 4: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({5: 'X'}, (2+0j)), ({2: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({4: 'X'}, (2+0j))]
... sparse_pauli_op = 
SparsePauliOp(['ZZIIII', 'ZIIIIZ', 'IZZIII', 'IIIIZZ', 'IIZZII', 'IIIZZI', 'XIIIII', 'IXIIII', 'IIIIIX', 'IIXIII', 'IIIXII', 'IIIIXI'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 2.+0.j, 2.+0.j, 2.+0.j,
 2.+0.j, 2.+0.j, 2.+0.j])

... Hamiltonian name = TFIM
[('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j)), ('XIIIII', (2+0j)), ('IXIIII', (2+0j)), ('IIIIIX', (2+0j)), ('IIXIII', (2+0j)), ('IIIXII', (2+0j)), ('IIIIXI', (2+0j))]



### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

In [4]:
# create the HamLib Simulation kernel, from the current Hamiltonian.
# We use a different method for kernel creation with HamLib than with custom Hamiltonians

########### create Trotterized evolution circuit for HamLib Hamiltonians

# create Hamiltonian evolution circuit from HamLib
if hamiltonian_name != '':
    init_state = "checkerboard"

    # this function reads the HamLib file content for the specified Hamiltonian
    #hamlib_simulation_kernel.get_hamiltonian_info(hamiltonian_name=hamiltonian_name)
    
    qc, bitstring = hamlib_simulation_kernel.HamiltonianSimulation(
        num_qubits=num_qubits, 
        ham_op=sparse_pauli_terms,
        K=K, t=t,
        init_state = init_state,
        append_measurements = False,
        method = 1, 
    )
    
########### create Trotterized evolution circuit for custom Hamiltonian
else:
    pass

print(f"... Hamiltonian name = {hamiltonian_name}")
print(pauli_terms)
print("")

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 10:
    print(qc)
    print("")


... Hamiltonian name = TFIM
[('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j)), ('XIIIII', (2+0j)), ('IXIIII', (2+0j)), ('IIIIIX', (2+0j)), ('IIXIII', (2+0j)), ('IIIXII', (2+0j)), ('IIIIXI', (2+0j))]

... Trotterized Circuit, K=1, t=0.05
     ┌───────────┐ ░                                                           »
q_0: ┤ U3(π,0,π) ├─░────────────────────────■───────────────────────────────■──»
     └───────────┘ ░                        │                               │  »
q_1: ──────────────░────────────────────────┼───────────────────────────────┼──»
     ┌───────────┐ ░                        │                               │  »
q_2: ┤ U3(π,0,π) ├─░────────────────────────┼───────────────────────────────┼──»
     └───────────┘ ░                        │                               │  »
q_3: ──────────────░────────────────────────┼───────■──────────────────■────┼──»
     ┌───────────┐ ░                        │

### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [5]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

# groups Pauli terms for quantum execution, optionally combining commuting terms into groups.
pauli_term_groups, pauli_str_list = observables.group_pauli_terms_for_execution(
        num_qubits, pauli_terms, use_commuting_groups)

# generate an array of circuits, one for each pauli_string in list
circuits = observables.create_circuits_for_pauli_terms(qc, num_qubits, pauli_str_list)
     
for circuit, group in list(zip(circuits, pauli_term_groups)):
    print(group)
    print(circuit)
    
N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile(circuits, backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()
    
    # Compute the total energy for the Hamiltonian
    term_contributions = {}
    total_energy = observables.calculate_expectation(num_qubits, results,
                                    circuits, pauli_term_groups,
                                    term_contributions=term_contributions)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")

('ZZIIII', (1+0j))
('ZIIIIZ', (1+0j))
('IZZIII', (1+0j))
('IIIIZZ', (1+0j))
('IIZZII', (1+0j))
('IIIZZI', (1+0j))
('XIIIII', (2+0j))
('IXIIII', (2+0j))
('IIIIIX', (2+0j))
('IIXIII', (2+0j))
('IIIXII', (2+0j))
('IIIIXI', (2+0j))
[('ZZIIII', (1+0j))]
        ┌───────────┐ ░                                                      »
   q_0: ┤ U3(π,0,π) ├─░────────────────────────■─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_1: ──────────────░────────────────────────┼─────────────────────────────»
        ┌───────────┐ ░                        │                             »
   q_2: ┤ U3(π,0,π) ├─░────────────────────────┼─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_3: ──────────────░────────────────────────┼───────■──────────────────■──»
        ┌───────────┐ ░                        │     ┌─┴─┐   ┌─────────┐┌─┴─┐»
   q_4: ┤ U3(π,0,π) ├─░───■─────────────

### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [6]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# groups Pauli terms for quantum execution, optionally combining commuting terms into groups.
pauli_term_groups, pauli_str_list = observables.group_pauli_terms_for_execution(
        num_qubits, pauli_terms, use_commuting_groups)

# generate an array of circuits, one for each pauli_string in list
circuits = observables.create_circuits_for_pauli_terms(qc, num_qubits, pauli_str_list)
     
for circuit, group in list(zip(circuits, pauli_term_groups)):
    print(group)
    print(circuit)
    
N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile(circuits, backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    term_contributions = {}
    total_energy = observables.calculate_expectation(num_qubits, results,
                                    circuits, pauli_term_groups,
                                    term_contributions=term_contributions)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")

... created pauli_term_groups:
Group 1:
  ZZIIII: (1+0j)
  ZIIIIZ: (1+0j)
  IZZIII: (1+0j)
  IIIIZZ: (1+0j)
  IIZZII: (1+0j)
  IIIZZI: (1+0j)
Group 2:
  XIIIII: (2+0j)
  IXIIII: (2+0j)
  IIIIIX: (2+0j)
  IIXIII: (2+0j)
  IIIXII: (2+0j)
  IIIIXI: (2+0j)
[('ZZIIII', (1+0j)), ('ZIIIIZ', (1+0j)), ('IZZIII', (1+0j)), ('IIIIZZ', (1+0j)), ('IIZZII', (1+0j)), ('IIIZZI', (1+0j))]
        ┌───────────┐ ░                                                      »
   q_0: ┤ U3(π,0,π) ├─░────────────────────────■─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_1: ──────────────░────────────────────────┼─────────────────────────────»
        ┌───────────┐ ░                        │                             »
   q_2: ┤ U3(π,0,π) ├─░────────────────────────┼─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_3: ──────────────░────────────────────────┼───────■─────────────────

### Compute expectation value of other observables.
Note that this only works for the Ising model.

In [7]:
# Define additional Hamiltonian terms for other Ising observables 
H_terms_spin_correlation = observables.swap_pauli_list([(0.2,'IIIIZZ'), (0.2,'IIIZZI'), (0.2,'IIZZII'), (0.2,'IZZIII'), (0.2,'ZZIIII')])
H_terms_magnetization = observables.swap_pauli_list([(1,'IIIIIZ'), (1,'IIIIZI'), (1,'IIIZII'), (1,'IIZIII'), (1,'IZIIII'), (1, 'ZIIIII')])

spin_correlation = observables.calculate_expectation_from_contributions(H_terms_spin_correlation, term_contributions)
print(spin_correlation)

magnetization = observables.calculate_expectation_from_contributions(H_terms_magnetization, term_contributions)
print(magnetization)


-0.962109375
WARN: term not found in term_contributions: IIIIIZ
WARN: term not found in term_contributions: IIIIZI
WARN: term not found in term_contributions: IIIZII
WARN: term not found in term_contributions: IIZIII
WARN: term not found in term_contributions: IZIIII
WARN: term not found in term_contributions: ZIIIII
0
